In [1]:
%%time

# This notebook was used to perform full order partial correlation on the climate related themes

# Import required libraries
import pandas as pd
import numpy as np
import pingouin as pg
  

# Load the time series data
df = pd.read_csv("ClustersTSParCorr.csv")
df.set_index('date',drop=True,inplace=True)
df.head()

Wall time: 25 s


,Energy,Carbon [emissions],Administration,Climate Science,Global Warming,Population & Economy,Plastic & Waste,Agriculture,Wildlife,Natural Catastrophes
date,,,,,,,,,,
29-02-2008,0.248198,0.046286,0.017461,0.023975,0.011779,0.040604,0.035338,0.045455,0.183204,0.006098
31-03-2008,0.227273,0.066667,0.009091,0.081818,0.042424,0.024242,0.051515,0.024242,0.184848,0.003030
30-04-2008,0.247257,0.056384,0.017650,0.079589,0.043961,0.046411,0.053606,0.074534,0.124206,0.014545
31-05-2008,0.359259,0.043834,0.022033,0.064013,0.026362,0.039738,0.032933,0.021645,0.144643,0.010900
30-06-2008,0.399834,0.050080,0.020334,0.059863,0.025003,0.040483,0.031256,0.060680,0.109026,0.009560


In [2]:
clusters = df.columns.to_list()
parCorr = pd.DataFrame()
for i in df.columns.to_list():
    for j in clusters:
        if i!=j:
            output = pg.partial_corr(data=df, x=f'{j}', y=f'{i}', covar=[k for k in clusters if k not in [i, j]]).round(3)
            output['-95%CI'] = output['CI95%'].map(lambda x: x[0])
            output['+95%CI'] = output['CI95%'].map(lambda x: x[1])
            output['y'] = i
            output['x'] = j            
            output.drop(['CI95%'],inplace=True,axis=1)
            parCorr = pd.concat([parCorr, output])

parCorr.head()    

,n,r,p-val,-95%CI,+95%CI,y,x
pearson,161,-0.357,0.000,-0.49,-0.21,Energy,Carbon [emissions]
pearson,161,-0.609,0.000,-0.70,-0.50,Energy,Administration
pearson,161,-0.657,0.000,-0.74,-0.56,Energy,Climate Science
pearson,161,-0.261,0.001,-0.40,-0.11,Energy,Global Warming
pearson,161,-0.658,0.000,-0.74,-0.56,Energy,Population & Economy


In [3]:
parCorr.to_csv("ClustersParCorrOutput.csv",index=False)